# Excution time measurements


In [1]:
cycles = 2

# Pandas

In [2]:
import pandas as pd
import sys
import os
import timeit

In [3]:
data1 = 'output/data1.csv'
data2 = 'output/data2.csv'



file_pathss = [
    'output/data1.csv',
    'output/data2.csv',
    'output/data3.csv',
    'output/data4.csv'
]


file_paths = [
    'output/data1.csv',
    'output/data2.csv',
    'output/data3.csv',
    'output/data4.csv',
    'output/data5.csv',
    'output/data6.csv',
    'output/data7.csv',
    'output/data8.csv',
    'output/data9.csv',
    'output/data10.csv',
    'output/data12.csv',
    'output/data13.csv',
    'output/data14.csv',
    'output/data15.csv',
]

bigger_files = [
    'output/data16.csv',
    'output/data17.csv',
    'output/data18.csv',
    'output/data19.csv',
    'output/data20.csv',
    'output/data21.csv',
    'output/data22.csv',
    'output/data23.csv',
    'output/data24.csv'
]


def average_time(myList):
    total_time_spark = 0
    for i in myList:
        total_time_spark += i
    avg_time = total_time_spark / cycles
    return avg_time

In [4]:
df1 = pd.read_csv(data1)


# Data Loading

In [127]:
def read_csv(x):
    df = pd.read_csv(x)
    return df

def read_time(dis):
    start_time = timeit.default_timer()
    dataframe = read_csv(dis)
    end_time = timeit.default_timer()
    return end_time - start_time


def measure_average_read_time(files, cycles=10):
    avg_times = []
    for file_path in files:
        timings_pandas = [read_time(file_path) for _ in range(cycles)]
        avg_time_pandas = average_time(timings_pandas)
        avg_times.append(avg_time_pandas)
    return avg_times

def run_loading(paths):
    avg_times = measure_average_read_time(paths)

    for i, avg_time in enumerate(avg_times, 1):
        print(f"Average read time for data_{i}: {avg_time:.5f} seconds")
        
run_loading(file_paths)


In [119]:
# Measuring lead time of bigger datasize - 6Gb - 15 GB
run_loading(bigger_files)

Average read time for data_1: 25.89928 seconds
Average read time for data_2: 39.33730 seconds
Average read time for data_3: 48.52571 seconds
Average read time for data_4: 60.61468 seconds
Average read time for data_5: 70.24155 seconds
Average read time for data_6: 76.79467 seconds
Average read time for data_7: 83.14161 seconds
Average read time for data_8: 94.47625 seconds
Average read time for data_9: 106.77703 seconds


# Filtering and Selection

In [136]:
test = pd.read_csv("output/data1.csv")

addresses = ["output/data1.csv"]

# we are filtering from a column "5.164162635803223" which are in values b/n and including 6 and 7.
def filter_and_select(filter_it):
    return filter_it.loc[(filter_it["5.164162635803223"] >= 6) & (filter_it["5.164162635803223"] <= 7)]

# Measurement - Filter_csv
def filter_time(dis):
    start_time = timeit.default_timer()
    dataframe = filter_and_select(dis)
    end_time = timeit.default_timer()
    return end_time - start_time

def measure_average_filter_time(files, cycles=10):
    avg_times = []
    for file_path in files:
        local_df = pd.read_csv(file_path)
        timings_pandas = [filter_time(local_df) for _ in range(cycles)]
        avg_time_pandas = average_time(timings_pandas)
        avg_times.append(avg_time_pandas)
    return avg_times

def run_filter(addresses):
    avg_times = measure_average_filter_time(addresses)

    for i, avg_time in enumerate(avg_times, 1):
        print(f"Average filter time for data-{i}: {avg_time:.5f} seconds")
        
run_filter(file_paths)


Average filter time for data-1: 0.00349 seconds
Average filter time for data-2: 0.00473 seconds
Average filter time for data-3: 0.00702 seconds
Average filter time for data-4: 0.00936 seconds
Average filter time for data-5: 0.01178 seconds
Average filter time for data-6: 0.01631 seconds
Average filter time for data-7: 0.01802 seconds
Average filter time for data-8: 0.01983 seconds
Average filter time for data-9: 0.03158 seconds
Average filter time for data-10: 0.03112 seconds
Average filter time for data-11: 0.08747 seconds
Average filter time for data-12: 0.18891 seconds
Average filter time for data-13: 0.72862 seconds
Average filter time for data-14: 1.95690 seconds


In [135]:
run_filter(bigger_files)

Average filter time for data-1: 1.55108 seconds
Average filter time for data-2: 2.82007 seconds
Average filter time for data-3: 4.15704 seconds
Average filter time for data-4: 5.44873 seconds
Average filter time for data-5: 7.53639 seconds
Average filter time for data-6: 8.85096 seconds
Average filter time for data-7: 13.64158 seconds
Average filter time for data-8: 55.19761 seconds
Average filter time for data-9: 56.10705 seconds


# Data Transformation

<h1>Spark</h1>

In [5]:
import pyspark
import timeit
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, mean, count, udf
import timeit
from pyspark.sql.types import StringType
from pyspark.sql import functions as F

# Data Loading

In [6]:
from pyspark.sql import SparkSession
import timeit

spark = SparkSession.builder.getOrCreate()

def read_csv(x):
    df = spark.read.option('header', 'true').csv(x, inferSchema=True)
    return df

def read_time(dis):
    start_time = timeit.default_timer()
    dataframe = read_csv(dis)
    end_time = timeit.default_timer()
    return end_time - start_time


def measure_average_read_time(files, cycles=10):
    avg_times = []
    for file_path in files:
        timings_spark = [read_time(file_path) for _ in range(cycles)]
        avg_time_spark = average_time(timings_spark)
        avg_times.append(avg_time_spark)
    return avg_times

def run_loading_spark(addresses):
    avg_times = measure_average_read_time(addresses)

    for i, avg_time in enumerate(avg_times, 1):
        print(f"Average read time for data_{i}: {avg_time:.5f} seconds")


23/06/08 12:14:39 WARN Utils: Your hostname, Merons-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.103 instead (on interface en0)
23/06/08 12:14:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/08 12:14:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [248]:
run_loading_spark(file_paths)

Average read time for data_1: 1.57070 seconds
Average read time for data_2: 2.80850 seconds
Average read time for data_3: 4.16848 seconds
Average read time for data_4: 5.90993 seconds
Average read time for data_5: 7.11664 seconds
Average read time for data_6: 8.18952 seconds
Average read time for data_7: 9.38334 seconds
Average read time for data_8: 10.13698 seconds
Average read time for data_9: 11.67882 seconds
Average read time for data_10: 13.04607 seconds
Average read time for data_11: 27.06401 seconds
Average read time for data_12: 40.34847 seconds
Average read time for data_13: 53.89822 seconds
Average read time for data_14: 67.64213 seconds


In [7]:
run_loading_spark(bigger_files)

Average read time for data_1: 104.29503 seconds
Average read time for data_2: 132.03032 seconds
Average read time for data_3: 149.14590 seconds
Average read time for data_4: 170.31554 seconds
Average read time for data_5: 233.77961 seconds
Average read time for data_6: 305.62885 seconds
Average read time for data_7: 249.09140 seconds
Average read time for data_8: 277.08775 seconds
Average read time for data_9: 341.47879 seconds


# Filtering data

# Trying to filter without action

In [8]:
from pyspark.sql import SparkSession
import timeit

spark = SparkSession.builder.getOrCreate()

# We are filtering from a column "5.164162635803223" which has values between and including 6 and 7.
def spark_filter_and_select(df):
    return df.filter((df["`5.164162635803223`"] >= 6) & (df["`5.164162635803223`"] <= 7))

# Measurement - Filter_csv
def spark_filter_time(df):
    start_time = timeit.default_timer()
    dataframe = spark_filter_and_select(df)
    end_time = timeit.default_timer()
    return end_time - start_time


def spark_measure_average_filter_time(files, cycles=10):
    avg_times = []
    for file_path in files:
        local_df = spark.read.option('header', 'true').csv(file_path, inferSchema=True)
        timings_spark = [spark_filter_time(local_df) for _ in range(cycles)]
        avg_time_spark = average_time(timings_spark)
        avg_times.append(avg_time_spark)
    return avg_times

def run_spark_filter(filesss):
    avg_times = spark_measure_average_filter_time(filesss)

    for i, avg_time in enumerate(avg_times, 1):
        if i >= 11:
            print(f"Average filter time for data-{i + 1}: {avg_time:.5f} seconds")
        print(f"Average filter time for data-{i}: {avg_time:.5f} seconds")

        
        
        


In [9]:
run_spark_filter(file_paths)

Average filter time for data-1: 0.08031 seconds
Average filter time for data-2: 0.04739 seconds
Average filter time for data-3: 0.04467 seconds
Average filter time for data-4: 0.04481 seconds
Average filter time for data-5: 0.04563 seconds
Average filter time for data-6: 0.04437 seconds
Average filter time for data-7: 0.04468 seconds
Average filter time for data-8: 0.04431 seconds
Average filter time for data-9: 0.04232 seconds
Average filter time for data-10: 0.04756 seconds
Average filter time for data-12: 0.04363 seconds
Average filter time for data-11: 0.04363 seconds
Average filter time for data-13: 0.04366 seconds
Average filter time for data-12: 0.04366 seconds
Average filter time for data-14: 0.04450 seconds
Average filter time for data-13: 0.04450 seconds
Average filter time for data-15: 0.04369 seconds
Average filter time for data-14: 0.04369 seconds


In [10]:
run_spark_filter(bigger_files)

Average filter time for data-1: 0.04396 seconds
Average filter time for data-2: 0.04485 seconds
Average filter time for data-3: 0.04469 seconds
Average filter time for data-4: 0.04246 seconds
Average filter time for data-5: 0.03135 seconds
Average filter time for data-6: 0.03210 seconds
Average filter time for data-7: 0.04341 seconds
Average filter time for data-8: 0.02842 seconds
Average filter time for data-9: 0.02528 seconds


# Data Transformation

# Trying to filter with an actual action

In [11]:
from pyspark.sql import SparkSession
import timeit

spark = SparkSession.builder.getOrCreate()

# We are filtering from a column "5.164162635803223" which has values between and including 6 and 7.
def spark_filter_and_select(df):
    return df.filter((df["`5.164162635803223`"] >= 6) & (df["`5.164162635803223`"] <= 7)).count()

# Measurement - Filter_csv
def spark_filter_time(df):
    start_time = timeit.default_timer()
    dataframe = spark_filter_and_select(df)
    end_time = timeit.default_timer()
    return end_time - start_time


def spark_measure_average_filter_time(files, cycles=10):
    avg_times = []
    for file_path in files:
        local_df = spark.read.option('header', 'true').csv(file_path, inferSchema=True)
        timings_spark = [spark_filter_time(local_df) for _ in range(cycles)]
        avg_time_spark = average_time(timings_spark)
        avg_times.append(avg_time_spark)
    return avg_times

def run_spark_filter(filesss):
    avg_times = spark_measure_average_filter_time(filesss)

    for i, avg_time in enumerate(avg_times, 1):
        if i >= 11:
            print(f"Average filter time for data-{i + 1}: {avg_time:.5f} seconds")
        print(f"Average filter time for data-{i}: {avg_time:.5f} seconds")


In [17]:
run_spark_filter(file_paths)

Average filter time for data-1: 0.59102 seconds
Average filter time for data-2: 1.02199 seconds
Average filter time for data-3: 1.89531 seconds
Average filter time for data-4: 4.22321 seconds
Average filter time for data-5: 5.73304 seconds
Average filter time for data-6: 6.51344 seconds
Average filter time for data-7: 8.08491 seconds
Average filter time for data-8: 9.38493 seconds
Average filter time for data-9: 10.75522 seconds
Average filter time for data-10: 11.46883 seconds
Average filter time for data-12: 15.49780 seconds
Average filter time for data-11: 15.49780 seconds
Average filter time for data-13: 16.79853 seconds
Average filter time for data-12: 16.79853 seconds
Average filter time for data-14: 18.25365 seconds
Average filter time for data-13: 18.25365 seconds
Average filter time for data-15: 27.86440 seconds
Average filter time for data-14: 27.86440 seconds


In [13]:
run_spark_filter(bigger_files)

Average filter time for data-1: 37.22346 seconds
Average filter time for data-2: 56.07037 seconds
Average filter time for data-3: 61.17328 seconds
Average filter time for data-4: 70.76579 seconds
Average filter time for data-5: 79.21383 seconds
Average filter time for data-6: 73.63960 seconds
Average filter time for data-7: 52.57020 seconds
Average filter time for data-8: 63.80649 seconds
Average filter time for data-9: 73.58435 seconds


# And for that reason, we also doing the results for pandas with counting of rows included

In [14]:
test = pd.read_csv("output/data1.csv")

addresses = ["output/data1.csv"]

# we are filtering from a column "5.164162635803223" which are in values b/n and including 6 and 7.
def filter_and_select(filter_it):
    return (filter_it.loc[(filter_it["5.164162635803223"] >= 6) & (filter_it["5.164162635803223"] <= 7)]).shape[0]

# Measurement - Filter_csv
def filter_time(dis):
    start_time = timeit.default_timer()
    dataframe = filter_and_select(dis)
    end_time = timeit.default_timer()
    return end_time - start_time

def measure_average_filter_time(files, cycles=10):
    avg_times = []
    for file_path in files:
        local_df = pd.read_csv(file_path)
        timings_pandas = [filter_time(local_df) for _ in range(cycles)]
        avg_time_pandas = average_time(timings_pandas)
        avg_times.append(avg_time_pandas)
    return avg_times

def run_filter(addresses):
    avg_times = measure_average_filter_time(addresses)

    for i, avg_time in enumerate(avg_times, 1):
        print(f"Pandas - Average filter time for data-{i}: {avg_time:.5f} seconds")
        


In [15]:
run_filter(file_paths)

Pandas - Average filter time for data-1: 0.01468 seconds
Pandas - Average filter time for data-2: 0.01710 seconds
Pandas - Average filter time for data-3: 0.02160 seconds
Pandas - Average filter time for data-4: 0.03065 seconds
Pandas - Average filter time for data-5: 0.04493 seconds
Pandas - Average filter time for data-6: 0.03973 seconds
Pandas - Average filter time for data-7: 0.05440 seconds
Pandas - Average filter time for data-8: 0.06208 seconds
Pandas - Average filter time for data-9: 0.06275 seconds
Pandas - Average filter time for data-10: 0.06771 seconds
Pandas - Average filter time for data-11: 0.16391 seconds
Pandas - Average filter time for data-12: 0.36188 seconds
Pandas - Average filter time for data-13: 0.59572 seconds
Pandas - Average filter time for data-14: 0.82604 seconds


In [16]:
run_filter(bigger_files)

Pandas - Average filter time for data-1: 0.90013 seconds
Pandas - Average filter time for data-2: 4.16967 seconds
Pandas - Average filter time for data-3: 5.31762 seconds
Pandas - Average filter time for data-4: 6.70582 seconds
Pandas - Average filter time for data-5: 11.42966 seconds
Pandas - Average filter time for data-6: 25.04504 seconds
Pandas - Average filter time for data-7: 17.40607 seconds
Pandas - Average filter time for data-8: 91.89603 seconds
Pandas - Average filter time for data-9: 139.21290 seconds
